In [ ]:
%cd ../code_

In [ ]:
from tableTesting import *
import pandas as pd
import os
import ast
from itertools import combinations
from typing import Any
import json
from collections import Counter

In [ ]:
def metric_pass_k(expected_answer, answer: Any, k: int) -> float:
    if not isinstance(answer, list):
        raise ValueError("Answer should be a list for pass@k metric.")

    # Convert the test.expect to a list if it's not already
    expected_values = expected_answer if isinstance(
        expected_answer, list) else [expected_answer]
    expected_values = list(map(str, expected_values))
    boolean_answers = [True if answer[i]
                       in expected_values else False for i in range(len(answer))]
    combinations_k = list(combinations(boolean_answers, k))
    passed_at_k = 0
    # Calculate the pass@k metric
    for comb in combinations_k:
        if any(comb):
            passed_at_k += 1
    pass_at_k_percentage = (passed_at_k / len(combinations_k))*100

    return pass_at_k_percentage

In [ ]:
def bool_results(expected_answer, answer: Any) -> float:
    if not isinstance(answer, list):
        raise ValueError("Answer should be a list for pass@k metric.")

    # Convert the test.expect to a list if it's not already
    expected_values = expected_answer if isinstance(
        expected_answer, list) else [expected_answer]
    expected_values = list(map(str, expected_values))
    if answer[0] in expected_values:
        return True
    return False

In [ ]:
def check(self, test, answer: Any) -> bool:
    if test.TestType in ["ColumnLookupTests", "RowLookupTests"]:
        if isinstance(test.expect, list):
            expected_values = test.expect if isinstance(
                test.expect, list) else [test.expect]
            expected_values = list(map(str, expected_values))
            matches = set(expected_values).intersection(set([answer]))
        else:
            print("Error: The expected Answer 'List' should be a list not string")
        return len(matches) > 0
    return str(test.expect) == answer

In [ ]:
benchmark_data_path = "../resources/BenchmarkDatasetClean/"
benchmark_dataset = os.listdir(benchmark_data_path)
benchmark_save_path = "../resources/BenchmarkResults31_8_2023/"
save_path_plot = "../resources/BenchmarkResults31_8_2023_plots/"
if not os.path.exists(save_path_plot):
    os.makedirs(save_path_plot)
sp = "../BenchmarkFinalResults_31_8_2023/"
if not os.path.exists(sp):
    os.makedirs(sp)
spa = "../BenchmarkFinalArrangedResults_31_8_2023/"
if not os.path.exists(spa):
    os.makedirs(spa)
benchmark_dataset = [
    data for data in benchmark_dataset if data.endswith(".csv")]
for bd in benchmark_dataset:
    try:
        pass_metrics = []
        t_F_bools = []
        print(bd)
        df = pd.read_csv(os.path.join(
            benchmark_save_path, bd, bd+"_output.csv"))
        print(df.shape)
        dirs = os.listdir(os.path.join(
            benchmark_save_path, bd, "cache_logger"))
        dirs = [d for d in dirs if d.endswith(".jsonl")]
        print(dirs)
        with open(os.path.join(benchmark_save_path, bd, "cache_logger", dirs[-1]), "r") as file:
            for line in file:
                line = ast.literal_eval(line.strip())
                if line.startswith("\"") and line.endswith("\""):
                    line = line[1:-1]
                data_ = json.loads((line))
                expected_ans = data_["test-expected-answer"]
                ans = data_["answer"]
                try:
                    pass_metrics.append(
                        [metric_pass_k(expected_ans, ans, k) for k in [1, 3, 5, 10, 15]])
                    t_F_bools.append(bool_results(expected_ans, ans))
                except Exception as err:
                    print(f"error {err} catched during metric calculation")
                    pass_metrics.append([None for k in [1, 3, 5, 10, 15]])
                    t_F_bools.append(False)
                    pass
        print(len(pass_metrics))
        df[['pass_1', 'pass_3', 'pass_5', 'pass_10', 'pass_15']] = pass_metrics
        df["T/F"] = t_F_bools
        print(df.columns)
        df.to_csv(os.path.join(benchmark_save_path, bd, bd +
                  "_output_revamped_metrics_with_bools.csv"), index=None)

    except Exception as Err:
        print("ERROR------------->", Err)
        pass